# Remember: CHANGE FILE's NAME BEFORE RUN THIS FILE

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import requests
import os
import datetime
import lxml
from bs4 import BeautifulSoup

In [2]:
def text_to_int(str):
    return str.replace('+','').replace(',','').replace('.','').replace(' ','').replace('N/A','')
def gettingData(date=0):
    url = 'https://www.worldometers.info/coronavirus/'
    req = requests.get(url)
    html_source = req.text

    # parsing it to beautiful soup
    soup = BeautifulSoup(html_source, 'html.parser')
    '''
    table = soup.find_all("tbody")
    table_rows = table[date*3].find_all('tr')
    '''
    table_rows = soup.findAll("table", {"id": date})[0].findAll('tr')[9:9+215]

    data = []
    another = []

    for i in range(len(table_rows)):
        row = []
        tag = table_rows[i].find(name = 'a', href = True)
        if tag is not None:
            row.append(tag.get_text()) #country
            td = table_rows[i].find_all(name='td')
            for j in range(2, 19, 1):
                if j != 15:
                    row.append(text_to_int(td[j].text.strip()))

            region = td[15].text.strip()
            if len(region): row.append(region)
            else: row.append("N/A")

            data.append(row)

        else:
            another.append(i) #no data at i-th row
    return data
def exportCSVfile(data, fileName):
    columns = ["Country", "Total Cases", "New Cases", "Total Deaths", "New Deaths", 
           "Total Recovered", "New Recovered", "Active Cases", "Serious, Critical", 
           "Tot Cases/1M pop", "Deaths/1M pop", "Total Tests", "Tests/1M pop", "Population", 
           "1 Case every X ppl", "1 Death every X ppl", "1 Test every X ppl", "Region"]
    df = pd.DataFrame(data, columns = columns)

    #convert object datatype to float/int datatypes, except country and region
    for col in df.columns[1:16]:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.fillna(0)
    df = df.sort_values("Total Cases", ascending=False).reset_index(drop=True)
    # Saving data to csv file
    df.to_csv(fileName)
    print("Sucessful")

In [3]:
fileName             = 'CoronaVirusDate20200918.csv'
fileName_yesterday   = 'CoronaVirusDate20200917.csv'
fileName_2daysbefore = 'CoronaVirusDate20200916.csv'

In [4]:
data_yesterday = gettingData('main_table_countries_yesterday')
exportCSVfile(data_yesterday, fileName_yesterday)

Sucessful


In [5]:
data_2daysbefore = gettingData('main_table_countries_yesterday2')
exportCSVfile(data_2daysbefore, fileName_2daysbefore)

Sucessful
